In [3]:
import pandas as pd
import glob
import os
import re
from pandas import ExcelWriter
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
#from fuzzy_expert.variable import FuzzyVariable
os.chdir("C:/Users/Mpho Bohloko/Desktop/science/DataBR")
df = pd.read_excel("C:/Users/Mpho Bohloko/Desktop/science/DataBR/Sample.xlsx",sheet_name="1. Original list received from ", engine='openpyxl')
#df1 = pd.read_excel("C:/Users/Mpho Bohloko/Desktop/science/DataBR/Sample.xlsx",sheet_name="1. Original list received from ", engine='openpyxl')
df.head()


,eC_ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,Country,Phone,Account ID,Account Name,Website
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,BR,11985305962,0013y00001cjwHH,Hospital São Camilo Unidade Ipiranga,http://www.hospitalsaocamilosp.org.br/unidades...
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,MX,7713808612,0013y00001Y0QYW,"Typhoon Sports Coalition, S.A.P.I. de C.V.",www.typhoonmx.com
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,FR,NaN,0013y00001Y0Vwz,Mobalpa,www.mobalpa.fr
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,FR,+33 2 5176 0100,0013y00001Y0PEO,Ocea,www.ocea.fr
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,GB,NaN,0013y00001cykLS,National Emergencies Trust,www.nationalemergenciestrust.org.uk


In [3]:
#checking missing data
df.isna().sum()

eC_ID               0
Contact ID          0
First Name          0
Last Name           0
Title               0
Email               0
Street Address     60
City               60
State/Province     67
Zip/Postal Code    60
Country             0
Phone              20
Account ID          0
Account Name        0
Website             0
dtype: int64

In [4]:
#Generating New Tittle variable, no logic information from Original dataset to generate answers seems to be manualy entered
df['New Tittle'] = 'NA'
#generate Change in Job level there is no logic information in original dataset
df['Change in Job Level'] = 'NA'

In [5]:
# Auto generate Job level variable without using drop downs. the note:string variables are in different languages matching does not work
df.loc[df.Title.isin(['Director','Assistant Director Fundraising',
                      'Gerente de Operaciones','Directeur en charge de Volkswagen Audi',
                     'Head of Business Engagement','Director - Un O & M Mexico',
                     'Head of Income Generation','Marketing Director, Trade Publishing','Development Manager',
                     'Directeur ventes et marketing digital','Director, Corporate Partnership Services',
                     'Director Premium Sales','Director, Information Technology',
                     'Director of Services','Digital Communication',
                     'Directeur des Systèmes dInformation','Head of Match Day and Brand Growth',
                     'Responsable du Service Prévention et Sécurité at Fondation ARHM (Action Recherch',
                     'Coordinateur commercial','Head of Operations','Directeur Service Client',
                     'Director, Procurement Services','Head of IT','Directeur commercial VN/VO','Directeur Back Office et Relation Clients',
                     'Directeur du service de relations de travail','Deputy Sales Director for Maritime Safety and Security']), "Job Level"] = "Director Level"
df.loc[df.Title.isin(['Group Chief Financial Officer','President & CEO Halifax Convention Centre Corporation','Directeur commercial VN/VO','Directeur Back Office et Relation Clients',
                     'Directeur général adjoint','Chief of Corporate Services',
                     'Chief Executive Officer & Publisher (Harlequin)',
                     'Chief Operations Officer & EOS Integrator','DIR FINANCIER','CEO','Fire Chief','Development Officer',
                     'Chief Operating Officer','Gerente General','MD/Chief Operating Officer (Directeur General)']), "Job Level"] = "C Level"
df.loc[df.Title.isin(['Manager - Territory Sales','Manager - Sales Engineering','Gerencia',
                     'Manager - Technical','Senior Manager - Sales Account','Fundraising Manager','Manager, Ecommerce and Email Marketing',
                      'Senior Supervisor of Logistics','Communications Officer',
                     'Assistant Manager, Digital Marketing','Senior Marketing Manager','Manager - Sales','Gerente De Ventas',
                      'Gerente de captação de recursos','Manager - Operations','Process and Operations Manager of the Biology & Anatomopathology IT System',
                      'Director de unidad de negocio','Commercial Manager','Fundraising Manager',
                      'Chef de projet e-services et transformation digitale','IT Manager','Sales Representative / Manager','Développeur et chargé de projets informatiques',
                     'Custumer Care Manager','3D Asset Supervisor','Coordenador de Parcerias e Novos Negócios','Cheffe de Projets Marketing Opérationnel']), "Job Level"] = "Manager Level"
df.loc[df.Title.isin(['Executivo de Novos Negócios e Inteligência de Mercado','Coordenadora do Serviço de Atendimento ao Cliente',
                      'Apresentadora','CG Animator','Ingenieur commercial','Editor']), "Job Level"] = "Staff Level"
df.loc[df.Title.isin(['VP Corporate Partnerships','Acting Vice President, Marketing','Vice-président',
                      'Vice President, Marketing and Communications','Executive Vice President - Marketing Digital','Executive Vice President, Digital',
                      'Vice President of Financial Operations','VP Human Resources','Vice President','Vice President - Human Resources','Vice President Operations',
                     'Senior Vice President- Client Relationship']), "Job Level"] = "Vice President Level"
df.head(10)

,eC_ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,Country,Phone,Account ID,Account Name,Website,New Tittle,Change in Job Level,Job Level
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,BR,11985305962,0013y00001cjwHH,Hospital São Camilo Unidade Ipiranga,http://www.hospitalsaocamilosp.org.br/unidades...,NA,NA,Staff Level
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,MX,7713808612,0013y00001Y0QYW,"Typhoon Sports Coalition, S.A.P.I. de C.V.",www.typhoonmx.com,NA,NA,Director Level
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,FR,NaN,0013y00001Y0Vwz,Mobalpa,www.mobalpa.fr,NA,NA,Manager Level
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,FR,+33 2 5176 0100,0013y00001Y0PEO,Ocea,www.ocea.fr,NA,NA,Director Level
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,GB,NaN,0013y00001cykLS,National Emergencies Trust,www.nationalemergenciestrust.org.uk,NA,NA,Director Level
5,eC_dhime_007,0033y00002ON1HvAAL,Mario,Carvalho,IT Manager,adilson.arcoverde@dreamm.com.br,"Rua do Paissandu, 58",Recife,NaN,50070-205,BR,+5581997364236,0013y00001cfjmN,Funda??o Chesf de Assist?ncia e Seguridade Social,www.fachesf.com.br,NA,NA,Manager Level
6,eC_dhime_009,0033y00002OGFUmAAP,Agnes,Marie-Egyptienne,MD/Chief Operating Officer (Directeur General),agnes.marie-egyptienne@arhm.fr,290 Route De Vienne - Bp 8252,Lyon,Auvergne-Rhône-Alpes,69355,FR,+33 4 3790 1010,0013y00001Y0VuC,Fondation A.R.H.M,www.arhm.fr,NA,NA,C Level
7,eC_dhime_010,0033y00002OwopyAAB,Abe,Hajar,VP Corporate Partnerships,ahajar@edmontonoilers.com,NaN,NaN,NaN,NaN,CA,(780) 414-4625,0013y00001Y0Sra,Oilers Entertainment Group,https://www.nhl.com/oilers,NA,NA,Vice President Level
8,eC_dhime_011,0033y00002P3blxAAB,Ai,Ogihara,Director,ai.ogihara@summitmx.com,Acceso III 15 A Benito Juarez,Santiago De Queretaro,NaN,76120,MX,52 44 2153 8100,0013y00001Y0Qbu,"Servilamina Summit Mexicana, S.A. de C.V.",www.summitmx.com,NA,NA,Director Level
9,eC_dhime_012,0033y00002P3blwAAB,Aida,Mendoza,Gerente General,aida@summitmx.com,Acceso III 15 A Benito Juarez,Santiago De Queretaro,NaN,76120,MX,52 44 2153 8100,0013y00001Y0Qbu,"Servilamina Summit Mexicana, S.A. de C.V.",www.summitmx.com,NA,NA,C Level


In [6]:
# auto Generate Job functions without usins  drop down, 
df.loc[df.Title.isin(['Executivo de Novos Negócios e Inteligência de Mercado','Deputy Sales Director for Maritime Safety and Security',
                      'VP Corporate Partnerships','Director','Manager - Territory Sales','Manager - Sales Engineering',
                      'Senior Manager - Sales Account','Business Affairs Manager','Director, Corporate Partnership Services',
                      'Director Premium Sales','Manager - Sales','Gerente De Ventas','Commercial Manager','Ingenieur commercial',
                      'Directeur commercial VN/VO']), "Job Function"] = "Sales"
df.loc[df.Title.isin(['Gerente de Operaciones','Directeur en charge de Volkswagen Audi','Director - Un O & M Mexico',
                      'Gerencia','Chief Operating Officer','Production Manager - Highway Thru Hell Season 12 (Discovery)',
                      'Vice-président','Operations Manager','Director of Services','Manager - Operations',
                      'Vice President','Process and Operations Manager of the Biology & Anatomopathology IT System','Head of Operations',
                      'Chief Operations Officer & EOS Integrator','Vice President Operations','Director']), "Job Function"] = "Operations"
df.loc[df.Title.isin(['Fundraising Manager','Cheffe de Projets Marketing Opérationnel','Assistant Director Fundraising',
                      'Marketing Director, Trade Publishing','Communications Officer','Vice President, Marketing and Communications',
                      'Communication','Product Marketing Manager - MIRA Books','Directeur ventes et marketing digital','Apresentadora',
                      'Executive Vice President - Marketing Digital','Executive Vice President, Digital','Design Communications Manager',
                      'Assistant Manager, Digital Marketing','Senior Marketing Manager','Digital Communication','Gerente de captação de recursos',
                      'CG Animator','Chef de projet e-services et transformation digitale']), "Job Function"] = "Marketing"
df.loc[df.Title.isin(['MD/Chief Operating Officer (Directeur General)','Gerente General','President & CEO Halifax Convention Centre Corporation',
                      'Directrice Au Conseil D Administration','Directeur général adjoint',
                      'Chief Executive Officer & Publisher (Harlequin)','CEO']), "Job Function"] = "Leadership (CEO/Founder)"
df.loc[df.Title.isin(['Manager - Technical','Director, Information Technology','Directeur des Systèmes d Information','Coordinateur commercial',
                      'Director de unidad de negocio','Head of IT']), "Job Function"] = "Information Technology"
df.loc[df.Title.isin(['VP Human Resources','Vice President - Human Resources']), "Job Function"] = "Human Resources"
df.loc[df.Title.isin(['Head of Income Generation']), "Job Function"] = "Healthcare / Environmental Sciences"
df.loc[df.Title.isin(['Chief Financial Officer','Group Chief Financial Officer','Vice President of Financial Operations','DIR FINANCIE']), "Job Function"] = "Finance"
df.loc[df.Title.isin(['Customer service manager','Coordenadora do Serviço de Atendimento ao Cliente','Head of Match Day and Brand Growth','Directeur Service Client',
                      'Directeur Back Office et Relation Clients']), "Job Function"] = "Customer Service"
df.loc[df.Title.isin(['Senior Supervisor of Logistics','Development Manager','Responsable du Service Prévention et Sécurité at Fondation ARHM (Action Recherch',
                      'Chief of Corporate Services','Director, Procurement Services']), "Job Function"] = "Business Services"
df.loc[df.Title.isin(['Senior Executive','Fundraising Manager']), "Job Function"] = "Other"
df.loc[df.Title.isin(['Head of Business Engagement']), "Job Function"] = "Strategy"
#view the data of 20 rows only
df.head(10)

,eC_ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,Country,Phone,Account ID,Account Name,Website,New Tittle,Change in Job Level,Job Level,Job Function
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,BR,11985305962,0013y00001cjwHH,Hospital São Camilo Unidade Ipiranga,http://www.hospitalsaocamilosp.org.br/unidades...,NA,NA,Staff Level,Sales
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,MX,7713808612,0013y00001Y0QYW,"Typhoon Sports Coalition, S.A.P.I. de C.V.",www.typhoonmx.com,NA,NA,Director Level,Operations
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,FR,NaN,0013y00001Y0Vwz,Mobalpa,www.mobalpa.fr,NA,NA,Manager Level,Marketing
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,FR,+33 2 5176 0100,0013y00001Y0PEO,Ocea,www.ocea.fr,NA,NA,Director Level,Sales
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,GB,NaN,0013y00001cykLS,National Emergencies Trust,www.nationalemergenciestrust.org.uk,NA,NA,Director Level,Marketing
5,eC_dhime_007,0033y00002ON1HvAAL,Mario,Carvalho,IT Manager,adilson.arcoverde@dreamm.com.br,"Rua do Paissandu, 58",Recife,NaN,50070-205,BR,+5581997364236,0013y00001cfjmN,Funda??o Chesf de Assist?ncia e Seguridade Social,www.fachesf.com.br,NA,NA,Manager Level,NaN
6,eC_dhime_009,0033y00002OGFUmAAP,Agnes,Marie-Egyptienne,MD/Chief Operating Officer (Directeur General),agnes.marie-egyptienne@arhm.fr,290 Route De Vienne - Bp 8252,Lyon,Auvergne-Rhône-Alpes,69355,FR,+33 4 3790 1010,0013y00001Y0VuC,Fondation A.R.H.M,www.arhm.fr,NA,NA,C Level,Leadership (CEO/Founder)
7,eC_dhime_010,0033y00002OwopyAAB,Abe,Hajar,VP Corporate Partnerships,ahajar@edmontonoilers.com,NaN,NaN,NaN,NaN,CA,(780) 414-4625,0013y00001Y0Sra,Oilers Entertainment Group,https://www.nhl.com/oilers,NA,NA,Vice President Level,Sales
8,eC_dhime_011,0033y00002P3blxAAB,Ai,Ogihara,Director,ai.ogihara@summitmx.com,Acceso III 15 A Benito Juarez,Santiago De Queretaro,NaN,76120,MX,52 44 2153 8100,0013y00001Y0Qbu,"Servilamina Summit Mexicana, S.A. de C.V.",www.summitmx.com,NA,NA,Director Level,Operations
9,eC_dhime_012,0033y00002P3blwAAB,Aida,Mendoza,Gerente General,aida@summitmx.com,Acceso III 15 A Benito Juarez,Santiago De Queretaro,NaN,76120,MX,52 44 2153 8100,0013y00001Y0Qbu,"Servilamina Summit Mexicana, S.A. de C.V.",www.summitmx.com,NA,NA,C Level,Leadership (CEO/Founder)


In [7]:
# Auto Generate Fuzzy matching by comparing First/Last Names Matching with emails  valid 80> and 90> above match all
df['LastName_Email'] = df.apply(lambda x: fuzz.partial_ratio(x['Last Name'], x['Email']), axis=1)
df['FirstName_Email'] = df.apply(lambda x: fuzz.partial_ratio(x['First Name'], x['Email']), axis=1)
#match emails with website 50% and above correct
df['Website_Email'] = df.apply(lambda x: fuzz.partial_ratio(x['Email'], x['Website']), axis=1)
# automate creating valid and invalid by matching email with website
df['Status'] = np.where((df.Website_Email > 49) & (df.LastName_Email > 0) | (df.FirstName_Email > 0), 'Valid', 'Invalid')
df['Match'] = np.where((df.FirstName_Email > 70) | (df.LastName_Email > 70), 'Valid', 'Mismatch between First/Last Name and Email')
df.head(10)

,eC_ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,...,Website,New Tittle,Change in Job Level,Job Level,Job Function,LastName_Email,FirstName_Email,Website_Email,Status,Match
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,...,http://www.hospitalsaocamilosp.org.br/unidades...,NA,NA,Staff Level,Sales,80,29,42,Valid,Valid
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,...,www.typhoonmx.com,NA,NA,Director Level,Operations,83,86,20,Valid,Valid
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,...,www.mobalpa.fr,NA,NA,Manager Level,Marketing,83,29,57,Valid,Valid
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,...,www.ocea.fr,NA,NA,Director Level,Sales,91,50,64,Valid,Valid
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,...,www.nationalemergenciestrust.org.uk,NA,NA,Director Level,Marketing,88,75,89,Valid,Valid
5,eC_dhime_007,0033y00002ON1HvAAL,Mario,Carvalho,IT Manager,adilson.arcoverde@dreamm.com.br,"Rua do Paissandu, 58",Recife,NaN,50070-205,...,www.fachesf.com.br,NA,NA,Manager Level,NaN,38,60,44,Valid,Mismatch between First/Last Name and Email
6,eC_dhime_009,0033y00002OGFUmAAP,Agnes,Marie-Egyptienne,MD/Chief Operating Officer (Directeur General),agnes.marie-egyptienne@arhm.fr,290 Route De Vienne - Bp 8252,Lyon,Auvergne-Rhône-Alpes,69355,...,www.arhm.fr,NA,NA,C Level,Leadership (CEO/Founder),88,80,64,Valid,Valid
7,eC_dhime_010,0033y00002OwopyAAB,Abe,Hajar,VP Corporate Partnerships,ahajar@edmontonoilers.com,NaN,NaN,NaN,NaN,...,https://www.nhl.com/oilers,NA,NA,Vice President Level,Sales,80,33,38,Valid,Valid
8,eC_dhime_011,0033y00002P3blxAAB,Ai,Ogihara,Director,ai.ogihara@summitmx.com,Acceso III 15 A Benito Juarez,Santiago De Queretaro,NaN,76120,...,www.summitmx.com,NA,NA,Director Level,Operations,86,50,75,Valid,Valid
9,eC_dhime_012,0033y00002P3blwAAB,Aida,Mendoza,Gerente General,aida@summitmx.com,Acceso III 15 A Benito Juarez,Santiago De Queretaro,NaN,76120,...,www.summitmx.com,NA,NA,C Level,Leadership (CEO/Founder),29,75,75,Valid,Valid


In [8]:
# Auto Generate Email status varial by matching First/Last Name vs Email
df['Email Status'] = np.where((df.FirstName_Email > 70) | (df.LastName_Email > 70), 'Given email verified', 'Given email invalid')
# generate Still at the company based on the matched email with names no other logic
df['Still at the company?'] = np.where((df.FirstName_Email > 15) | (df.LastName_Email > 15), 'Yes', 'No')
df.head(10)

,eC_ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,...,Change in Job Level,Job Level,Job Function,LastName_Email,FirstName_Email,Website_Email,Status,Match,Email Status,Still at the company?
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,...,NA,Staff Level,Sales,80,29,42,Valid,Valid,Given email verified,Yes
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,...,NA,Director Level,Operations,83,86,20,Valid,Valid,Given email verified,Yes
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,...,NA,Manager Level,Marketing,83,29,57,Valid,Valid,Given email verified,Yes
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,...,NA,Director Level,Sales,91,50,64,Valid,Valid,Given email verified,Yes
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,...,NA,Director Level,Marketing,88,75,89,Valid,Valid,Given email verified,Yes
5,eC_dhime_007,0033y00002ON1HvAAL,Mario,Carvalho,IT Manager,adilson.arcoverde@dreamm.com.br,"Rua do Paissandu, 58",Recife,NaN,50070-205,...,NA,Manager Level,NaN,38,60,44,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes
6,eC_dhime_009,0033y00002OGFUmAAP,Agnes,Marie-Egyptienne,MD/Chief Operating Officer (Directeur General),agnes.marie-egyptienne@arhm.fr,290 Route De Vienne - Bp 8252,Lyon,Auvergne-Rhône-Alpes,69355,...,NA,C Level,Leadership (CEO/Founder),88,80,64,Valid,Valid,Given email verified,Yes
7,eC_dhime_010,0033y00002OwopyAAB,Abe,Hajar,VP Corporate Partnerships,ahajar@edmontonoilers.com,NaN,NaN,NaN,NaN,...,NA,Vice President Level,Sales,80,33,38,Valid,Valid,Given email verified,Yes
8,eC_dhime_011,0033y00002P3blxAAB,Ai,Ogihara,Director,ai.ogihara@summitmx.com,Acceso III 15 A Benito Juarez,Santiago De Queretaro,NaN,76120,...,NA,Director Level,Operations,86,50,75,Valid,Valid,Given email verified,Yes
9,eC_dhime_012,0033y00002P3blwAAB,Aida,Mendoza,Gerente General,aida@summitmx.com,Acceso III 15 A Benito Juarez,Santiago De Queretaro,NaN,76120,...,NA,C Level,Leadership (CEO/Founder),29,75,75,Valid,Valid,Given email verified,Yes


In [9]:
# renaming variables names
df = df.rename(columns = {'Street Address': 'Contact Address', 'City': 'Contact City', 'Zip/Postal Code' : 'Contact Zip Code',
                          'Country' : 'Contact Country', 'State/Province' : 'Contact State'})
#df = df[['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date',
     #  'Purchase Address']]
df

,eC_ID,Contact ID,First Name,Last Name,Title,Email,Contact Address,Contact City,Contact State,Contact Zip Code,...,Change in Job Level,Job Level,Job Function,LastName_Email,FirstName_Email,Website_Email,Status,Match,Email Status,Still at the company?
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,...,NA,Staff Level,Sales,80,29,42,Valid,Valid,Given email verified,Yes
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,...,NA,Director Level,Operations,83,86,20,Valid,Valid,Given email verified,Yes
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,...,NA,Manager Level,Marketing,83,29,57,Valid,Valid,Given email verified,Yes
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,...,NA,Director Level,Sales,91,50,64,Valid,Valid,Given email verified,Yes
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,...,NA,Director Level,Marketing,88,75,89,Valid,Valid,Given email verified,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,eC_dhime_334,0033y00002NTZk2AAH,Carlos,Fernandes,Mr.,projetos@academiaperolasnegras.com.br,"Avenida Antão Bernardes, 3000, bairro de Forta...",Paty do Alferes,NaN,26950-000,...,NA,NaN,NaN,44,33,78,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes
96,eC_dhime_372,0033y00002JaQdqAAF,sara,Dramé,Custumer Care Manager,sara@anybuddyapp.com,EURATECHNOLOGIES,Lille,FR-59,59000,...,NA,Manager Level,NaN,40,100,79,Valid,Valid,Given email verified,Yes
97,eC_dhime_386,0033y00002PNFcIAAX,Sébastien,Robert,Directeur du service de relations de travail,srobert@aptsq.com,NaN,NaN,NaN,NaN,...,NA,Director Level,NaN,83,22,69,Valid,Valid,Given email verified,Yes
98,eC_dhime_402,0033y00002PQGxpAAH,Tristan,Goodes,3D Asset Supervisor,tgoodes@atomiccartoons.com,123 7th Ave W,Vancouver,BC,V5Y 1L8,...,NA,Manager Level,NaN,83,29,23,Valid,Valid,Given email verified,Yes


In [10]:
# create clean website script
df['Clean Websites'] = df['Website'].str[4:]
df['Clean Websites'] = df['Clean Websites'].str.replace('s://www.', '', regex=True)
df['Clean Websites'] = df['Clean Websites'].str.replace('://www.', '', regex=True)
#df = df['Clean Websites'] = df['Clean Websites'].str.replace('s://', '', regex=True)
#viewing the dataset
df.head(10)

,eC_ID,Contact ID,First Name,Last Name,Title,Email,Contact Address,Contact City,Contact State,Contact Zip Code,...,Job Level,Job Function,LastName_Email,FirstName_Email,Website_Email,Status,Match,Email Status,Still at the company?,Clean Websites
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,...,Staff Level,Sales,80,29,42,Valid,Valid,Given email verified,Yes,hospitalsaocamilosp.org.br/unidades/ipiranga
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,...,Director Level,Operations,83,86,20,Valid,Valid,Given email verified,Yes,typhoonmx.com
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,...,Manager Level,Marketing,83,29,57,Valid,Valid,Given email verified,Yes,mobalpa.fr
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,...,Director Level,Sales,91,50,64,Valid,Valid,Given email verified,Yes,ocea.fr
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,...,Director Level,Marketing,88,75,89,Valid,Valid,Given email verified,Yes,nationalemergenciestrust.org.uk
5,eC_dhime_007,0033y00002ON1HvAAL,Mario,Carvalho,IT Manager,adilson.arcoverde@dreamm.com.br,"Rua do Paissandu, 58",Recife,NaN,50070-205,...,Manager Level,NaN,38,60,44,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,fachesf.com.br
6,eC_dhime_009,0033y00002OGFUmAAP,Agnes,Marie-Egyptienne,MD/Chief Operating Officer (Directeur General),agnes.marie-egyptienne@arhm.fr,290 Route De Vienne - Bp 8252,Lyon,Auvergne-Rhône-Alpes,69355,...,C Level,Leadership (CEO/Founder),88,80,64,Valid,Valid,Given email verified,Yes,arhm.fr
7,eC_dhime_010,0033y00002OwopyAAB,Abe,Hajar,VP Corporate Partnerships,ahajar@edmontonoilers.com,NaN,NaN,NaN,NaN,...,Vice President Level,Sales,80,33,38,Valid,Valid,Given email verified,Yes,nhl.com/oilers
8,eC_dhime_011,0033y00002P3blxAAB,Ai,Ogihara,Director,ai.ogihara@summitmx.com,Acceso III 15 A Benito Juarez,Santiago De Queretaro,NaN,76120,...,Director Level,Operations,86,50,75,Valid,Valid,Given email verified,Yes,summitmx.com
9,eC_dhime_012,0033y00002P3blwAAB,Aida,Mendoza,Gerente General,aida@summitmx.com,Acceso III 15 A Benito Juarez,Santiago De Queretaro,NaN,76120,...,C Level,Leadership (CEO/Founder),29,75,75,Valid,Valid,Given email verified,Yes,summitmx.com


In [11]:
df['Clean Websites'] = df['Clean Websites'].str.replace('s://', '', regex=True)
df

,eC_ID,Contact ID,First Name,Last Name,Title,Email,Contact Address,Contact City,Contact State,Contact Zip Code,...,Job Level,Job Function,LastName_Email,FirstName_Email,Website_Email,Status,Match,Email Status,Still at the company?,Clean Websites
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,...,Staff Level,Sales,80,29,42,Valid,Valid,Given email verified,Yes,hospitalsaocamilosp.org.br/unidades/ipiranga
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,...,Director Level,Operations,83,86,20,Valid,Valid,Given email verified,Yes,typhoonmx.com
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,...,Manager Level,Marketing,83,29,57,Valid,Valid,Given email verified,Yes,mobalpa.fr
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,...,Director Level,Sales,91,50,64,Valid,Valid,Given email verified,Yes,ocea.fr
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,...,Director Level,Marketing,88,75,89,Valid,Valid,Given email verified,Yes,nationalemergenciestrust.org.uk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,eC_dhime_334,0033y00002NTZk2AAH,Carlos,Fernandes,Mr.,projetos@academiaperolasnegras.com.br,"Avenida Antão Bernardes, 3000, bairro de Forta...",Paty do Alferes,NaN,26950-000,...,NaN,NaN,44,33,78,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,://academiaperolasnegras.org
96,eC_dhime_372,0033y00002JaQdqAAF,sara,Dramé,Custumer Care Manager,sara@anybuddyapp.com,EURATECHNOLOGIES,Lille,FR-59,59000,...,Manager Level,NaN,40,100,79,Valid,Valid,Given email verified,Yes,anybuddyapp.com
97,eC_dhime_386,0033y00002PNFcIAAX,Sébastien,Robert,Directeur du service de relations de travail,srobert@aptsq.com,NaN,NaN,NaN,NaN,...,Director Level,NaN,83,22,69,Valid,Valid,Given email verified,Yes,aptsq.com
98,eC_dhime_402,0033y00002PQGxpAAH,Tristan,Goodes,3D Asset Supervisor,tgoodes@atomiccartoons.com,123 7th Ave W,Vancouver,BC,V5Y 1L8,...,Manager Level,NaN,83,29,23,Valid,Valid,Given email verified,Yes,thunderbird.tv


In [18]:
# auto Create 5. Delivery Tab 1 Validated .valid data using Email match with website dataset
dfr = df.loc[(df['Status'] == 'Valid') | (df['Match'] == 'Valid')]

#First 20 rows display
dfr

,eC_ID,Contact ID,First Name,Last Name,Title,Email,Contact Address,Contact City,Contact State,Contact Zip Code,...,Job Level,Job Function,LastName_Email,FirstName_Email,Website_Email,Status,Match,Email Status,Still at the company?,Clean Websites
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,...,Staff Level,Sales,80,29,42,Valid,Valid,Given email verified,Yes,hospitalsaocamilosp.org.br/unidades/ipiranga
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,...,Director Level,Operations,83,86,20,Valid,Valid,Given email verified,Yes,typhoonmx.com
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,...,Manager Level,Marketing,83,29,57,Valid,Valid,Given email verified,Yes,mobalpa.fr
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,...,Director Level,Sales,91,50,64,Valid,Valid,Given email verified,Yes,ocea.fr
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,...,Director Level,Marketing,88,75,89,Valid,Valid,Given email verified,Yes,nationalemergenciestrust.org.uk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,eC_dhime_334,0033y00002NTZk2AAH,Carlos,Fernandes,Mr.,projetos@academiaperolasnegras.com.br,"Avenida Antão Bernardes, 3000, bairro de Forta...",Paty do Alferes,NaN,26950-000,...,NaN,NaN,44,33,78,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,://academiaperolasnegras.org
96,eC_dhime_372,0033y00002JaQdqAAF,sara,Dramé,Custumer Care Manager,sara@anybuddyapp.com,EURATECHNOLOGIES,Lille,FR-59,59000,...,Manager Level,NaN,40,100,79,Valid,Valid,Given email verified,Yes,anybuddyapp.com
97,eC_dhime_386,0033y00002PNFcIAAX,Sébastien,Robert,Directeur du service de relations de travail,srobert@aptsq.com,NaN,NaN,NaN,NaN,...,Director Level,NaN,83,22,69,Valid,Valid,Given email verified,Yes,aptsq.com
98,eC_dhime_402,0033y00002PQGxpAAH,Tristan,Goodes,3D Asset Supervisor,tgoodes@atomiccartoons.com,123 7th Ave W,Vancouver,BC,V5Y 1L8,...,Manager Level,NaN,83,29,23,Valid,Valid,Given email verified,Yes,thunderbird.tv


In [19]:
# valid data using Email match with website
df1 = df[df["Status"] == 'Valid'] 
#mismatch email and website
dfm = df[df["Status"] == 'Invalid']
# mismatch Names and email dataset
dfm1 = df[df["Match"] == 'Mismatch between First/Last Name and Email']
dfm

,eC_ID,Contact ID,First Name,Last Name,Title,Email,Contact Address,Contact City,Contact State,Contact Zip Code,...,Job Level,Job Function,LastName_Email,FirstName_Email,Website_Email,Status,Match,Email Status,Still at the company?,Clean Websites


In [20]:
#matched names vs Email
df2 = df[df['Match'] == 'Valid']
# mismatch Names and email dataset
dfm2 = df[df['Match'] == 'Mismatch between First/Last Name and Email']
dfm2

,eC_ID,Contact ID,First Name,Last Name,Title,Email,Contact Address,Contact City,Contact State,Contact Zip Code,...,Job Level,Job Function,LastName_Email,FirstName_Email,Website_Email,Status,Match,Email Status,Still at the company?,Clean Websites
5,eC_dhime_007,0033y00002ON1HvAAL,Mario,Carvalho,IT Manager,adilson.arcoverde@dreamm.com.br,"Rua do Paissandu, 58",Recife,NaN,50070-205,...,Manager Level,NaN,38,60,44,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,fachesf.com.br
40,eC_dhime_047,0033y00002OBQmWAAX,Juliana Lívia,Bortolin Sotero,Apresentadora,bortolin@lbv.org.br,NaN,NaN,SP,NaN,...,Staff Level,Marketing,60,31,42,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,lbv.org/
54,eC_dhime_062,0033y00002P3akXAAR,Carlos Martinez,Monroy,Gerente De Ventas,carlos@gcuetara.com.mx,"Tochtili No 342, Colonia Fracc Industrial San ...",Mexico City,NaN,2760,...,Manager Level,Sales,17,47,79,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,gcuetara.com.mx
59,eC_dhime_068,0033y00002PNFhSAAX,Caroline,Simoneau,Directrice Au Conseil D'Administration,cbeauregard@aptsq.com,NaN,NaN,NaN,NaN,...,NaN,NaN,38,25,69,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,aptsq.com
70,eC_dhime_079,0033y00002POYxsAAH,Irene,Liu,CG Animator,cliu@atomiccartoons.com,NaN,NaN,NaN,NaN,...,Staff Level,Marketing,67,20,20,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,thunderbird.tv
71,eC_dhime_080,0033y00002KWn0WAAT,David,Clay,Commercial Manager,commercial@scunthorpe-united.co.uk,NaN,NaN,NaN,NaN,...,Manager Level,Sales,25,40,85,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,scunthorpe-united.co.uk
87,eC_dhime_167,0033y00002OGyC6AAL,Gustavo,Lima Olmos Ferraz,Auxiliar de escritório,guolmos06@gmail.com,NaN,NaN,SP,NaN,...,NaN,NaN,29,29,32,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,hospitalsaocamilosp.org.br/unidades/ipiranga
95,eC_dhime_334,0033y00002NTZk2AAH,Carlos,Fernandes,Mr.,projetos@academiaperolasnegras.com.br,"Avenida Antão Bernardes, 3000, bairro de Forta...",Paty do Alferes,NaN,26950-000,...,NaN,NaN,44,33,78,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,://academiaperolasnegras.org


In [21]:
# 5. Delivery Tab 4 Mismatching Names vs Emails and Emails domain vs websites domain
dfm4=df.loc[(df['Status'] == 'Invalid') | (df['Match'] == 'Mismatch between First/Last Name and Email')]
dfm4


,eC_ID,Contact ID,First Name,Last Name,Title,Email,Contact Address,Contact City,Contact State,Contact Zip Code,...,Job Level,Job Function,LastName_Email,FirstName_Email,Website_Email,Status,Match,Email Status,Still at the company?,Clean Websites
5,eC_dhime_007,0033y00002ON1HvAAL,Mario,Carvalho,IT Manager,adilson.arcoverde@dreamm.com.br,"Rua do Paissandu, 58",Recife,NaN,50070-205,...,Manager Level,NaN,38,60,44,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,fachesf.com.br
40,eC_dhime_047,0033y00002OBQmWAAX,Juliana Lívia,Bortolin Sotero,Apresentadora,bortolin@lbv.org.br,NaN,NaN,SP,NaN,...,Staff Level,Marketing,60,31,42,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,lbv.org/
54,eC_dhime_062,0033y00002P3akXAAR,Carlos Martinez,Monroy,Gerente De Ventas,carlos@gcuetara.com.mx,"Tochtili No 342, Colonia Fracc Industrial San ...",Mexico City,NaN,2760,...,Manager Level,Sales,17,47,79,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,gcuetara.com.mx
59,eC_dhime_068,0033y00002PNFhSAAX,Caroline,Simoneau,Directrice Au Conseil D'Administration,cbeauregard@aptsq.com,NaN,NaN,NaN,NaN,...,NaN,NaN,38,25,69,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,aptsq.com
70,eC_dhime_079,0033y00002POYxsAAH,Irene,Liu,CG Animator,cliu@atomiccartoons.com,NaN,NaN,NaN,NaN,...,Staff Level,Marketing,67,20,20,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,thunderbird.tv
71,eC_dhime_080,0033y00002KWn0WAAT,David,Clay,Commercial Manager,commercial@scunthorpe-united.co.uk,NaN,NaN,NaN,NaN,...,Manager Level,Sales,25,40,85,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,scunthorpe-united.co.uk
87,eC_dhime_167,0033y00002OGyC6AAL,Gustavo,Lima Olmos Ferraz,Auxiliar de escritório,guolmos06@gmail.com,NaN,NaN,SP,NaN,...,NaN,NaN,29,29,32,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,hospitalsaocamilosp.org.br/unidades/ipiranga
95,eC_dhime_334,0033y00002NTZk2AAH,Carlos,Fernandes,Mr.,projetos@academiaperolasnegras.com.br,"Avenida Antão Bernardes, 3000, bairro de Forta...",Paty do Alferes,NaN,26950-000,...,NaN,NaN,44,33,78,Valid,Mismatch between First/Last Name and Email,Given email invalid,Yes,://academiaperolasnegras.org


In [26]:
#Final 5. Delivery Tab 1 validated
dfs1 = dfr[['eC_ID', 'Contact ID', 'Account ID', 'Account Name', 'Phone', 'First Name', 'Last Name', 'Clean Websites','Title','New Tittle', 'Job Level', 'Job Function',
         'Change in Job Level', 'Email Status', 'Still at the company?', 'Contact Address', 'Contact State',
         'Contact Zip Code', 'Contact Country','Contact City', 'Website','Status']]
#View 20 records Final 5. Delivery Tab 1 validated
dfs1.head(20)

,eC_ID,Contact ID,Account ID,Account Name,Phone,First Name,Last Name,Clean Websites,Title,New Tittle,...,Change in Job Level,Email Status,Still at the company?,Contact Address,Contact State,Contact Zip Code,Contact Country,Contact City,Website,Status
0,eC_dhime_001,0033y00002OH1jhAAD,0013y00001cjwHH,Hospital São Camilo Unidade Ipiranga,11985305962,Adriano,Alves,hospitalsaocamilosp.org.br/unidades/ipiranga,Executivo de Novos Negócios e Inteligência de ...,NA,...,NA,Given email verified,Yes,NaN,SP,NaN,BR,NaN,http://www.hospitalsaocamilosp.org.br/unidades...,Valid
1,eC_dhime_003,0033y00002N5F0rAAF,0013y00001Y0QYW,"Typhoon Sports Coalition, S.A.P.I. de C.V.",7713808612,Abraham,Vargas,typhoonmx.com,Gerente de Operaciones,NA,...,NA,Given email verified,Yes,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,11950,MX,Ciudad de Mexico,www.typhoonmx.com,Valid
2,eC_dhime_004,0033y00002Q1TmMAAV,0013y00001Y0Vwz,Mobalpa,NaN,Auriane,Breton,mobalpa.fr,Cheffe de Projets Marketing Opérationnel,NA,...,NA,Given email verified,Yes,NaN,NaN,NaN,FR,NaN,www.mobalpa.fr,Valid
3,eC_dhime_005,0033y00002PnGyCAAV,0013y00001Y0PEO,Ocea,+33 2 5176 0100,Amaury,Decacqueray,ocea.fr,Deputy Sales Director for Maritime Safety and ...,NA,...,NA,Given email verified,Yes,"Croisement, Avenue de Penhoet",Pays de la Loire,44600,FR,Saint-Nazaire,www.ocea.fr,Valid
4,eC_dhime_006,0033y00002NcyjsAAB,0013y00001cykLS,National Emergencies Trust,NaN,Adil,Husseini,nationalemergenciestrust.org.uk,Assistant Director Fundraising,NA,...,NA,Given email verified,Yes,NaN,NaN,NaN,GB,NaN,www.nationalemergenciestrust.org.uk,Valid
5,eC_dhime_007,0033y00002ON1HvAAL,0013y00001cfjmN,Funda??o Chesf de Assist?ncia e Seguridade Social,+5581997364236,Mario,Carvalho,fachesf.com.br,IT Manager,NA,...,NA,Given email invalid,Yes,"Rua do Paissandu, 58",NaN,50070-205,BR,Recife,www.fachesf.com.br,Valid
6,eC_dhime_009,0033y00002OGFUmAAP,0013y00001Y0VuC,Fondation A.R.H.M,+33 4 3790 1010,Agnes,Marie-Egyptienne,arhm.fr,MD/Chief Operating Officer (Directeur General),NA,...,NA,Given email verified,Yes,290 Route De Vienne - Bp 8252,Auvergne-Rhône-Alpes,69355,FR,Lyon,www.arhm.fr,Valid
7,eC_dhime_010,0033y00002OwopyAAB,0013y00001Y0Sra,Oilers Entertainment Group,(780) 414-4625,Abe,Hajar,nhl.com/oilers,VP Corporate Partnerships,NA,...,NA,Given email verified,Yes,NaN,NaN,NaN,CA,NaN,https://www.nhl.com/oilers,Valid
8,eC_dhime_011,0033y00002P3blxAAB,0013y00001Y0Qbu,"Servilamina Summit Mexicana, S.A. de C.V.",52 44 2153 8100,Ai,Ogihara,summitmx.com,Director,NA,...,NA,Given email verified,Yes,Acceso III 15 A Benito Juarez,NaN,76120,MX,Santiago De Queretaro,www.summitmx.com,Valid
9,eC_dhime_012,0033y00002P3blwAAB,0013y00001Y0Qbu,"Servilamina Summit Mexicana, S.A. de C.V.",52 44 2153 8100,Aida,Mendoza,summitmx.com,Gerente General,NA,...,NA,Given email verified,Yes,Acceso III 15 A Benito Juarez,NaN,76120,MX,Santiago De Queretaro,www.summitmx.com,Valid


In [25]:
#exporting combined excel files for google sheets
with pd.ExcelWriter('mult_sheets_1.xlsx') as writer1:
    dfs1.to_excel(writer1, sheet_name = 'Delivery_Tab1', index = False)
    dfm4.to_excel(writer1, sheet_name = 'Delivery_Tab4', index = False)